# 定数項を含む単純回帰モデル
真のデータ発生過程が次式で与えられていると仮定する．
$$
y_i = \alpha_0 + \beta_0x_i + u_i,~~~~ i = 1,\ldots,n
$$
ここで定数項を含む単回帰モデルを考える．
$$
y_i = \alpha + \beta x_i + u_i,~~~~ i = 1,\ldots,n
$$
ただし，誤差項 $u$ について，次の仮定を課す．
$$
\begin{align*}
&\text{E}(u_i) = 0~~\forall i, \\
&\text{E}(u_i^2) = \sigma_u^2~~\forall i, \\
&\text{E}(u_iu_j) = 0~~(i \neq j), \\
&u_i\overset{i.i.d}{\sim} N(0, \sigma_u^2)~~\forall i.
\end{align*}
$$

最小二乗推定量 $\hat{\alpha},\hat{\beta}$ は
$$
\hat{\alpha} = \bar{y} - \hat{\beta}\bar{x},~~~~
\hat{\beta} = \frac{\sum_i (x_i-\bar{x})(y_i-\bar{y})}{\sum_i (x_i-\bar{x})^2}
$$
で与えられる．ただし，$\bar{x}, \bar{y}$ は標本平均である．OLS推定量 $\hat{\alpha}, \hat{\beta}$ は，次の正規分布に従うことが知られている．
$$
\hat{\alpha} \sim N\left\{\alpha_0,\sigma_u^2\left(\frac{1}{n}+\frac{\bar{x}^2}{s_x}\right)\right\},~~~~
\hat{\beta} \sim N\left\{\beta_0, \frac{\sigma_u^2}{s_x}\right\}~~~~
\text{where}~~s_x \equiv \sum_i(x_i - \bar{x})^2
$$
実際のデータ分析では，誤差項の分散 $\sigma_u^2$ は未知なので，データから推定するしかない．誤差項の分散の推定量は次式で与えられる．
$$
\hat{\sigma_u^2} = \frac{1}{n-2}\sum_i(y_i - \hat{\alpha}-\hat{\beta}x_i)^2
$$
この推定量を使うと，OLS推定量 $\hat{\alpha}, \hat{\beta}$ の標準誤差（標準偏差の推定量）は
$$
\widehat{se}_{\alpha} = \sqrt{\hat{\sigma_u}^2\left(\frac{1}{n}+\frac{\bar{x}^2}{s_x}\right)},~~~~
\widehat{se}_{\beta} = \sqrt{\frac{\hat{\sigma_u}^2}{s_x}}
$$
となる．これらの推定量を使うと分析者のパラメータについての仮説 $\alpha_0 = \alpha^*,\beta_0 = \beta^*$ の下で，$\hat{\alpha}, \hat{\beta}$ のt統計量はそれぞれ，
$$
\hat{t}_\alpha = \frac{\hat{\alpha}-\alpha^*}{\widehat{se}_{\alpha}},~~~~
\hat{t}_\beta = \frac{\hat{\beta}-\beta^*}{\widehat{se}_{\beta}}
$$
で計算できる．t統計量は自由度 $n-2$ のt分布に従うので，この事実をもとにして信頼区間や棄却域を計算できる．

## t検定の信頼区間
$\beta$ の信頼区間を求める．t統計量は自由度 $n-2$ のt分布に従う．
$$
t_\beta = \frac{\hat{\beta}-\beta_0}{\widehat{se}_{\beta}} \sim t_{n-2}
$$
この性質から信頼度 $1-a$ の信頼区間を求めることができる．臨界値 $r_{a}$の定義から
$$
\begin{align}
1-a &= \Pr\left(\left|\frac{\hat{\beta}-\beta_0}{\widehat{se}_{\beta}}\right| \leq r_a \right) \\
&= \Pr\left(-r_a \leq \frac{\hat{\beta}-\beta_0}{\widehat{se}_{\beta}} \leq r_a \right) \\
&=\Pr\left(\hat{\beta}-r_a\widehat{se}_{\beta} \leq \beta_0 \leq \hat{\beta}+r_a\widehat{se}_{\beta} \right)
\end{align}
$$
と変形でき，この $\hat{\beta}-r_a\widehat{se}_{\beta} \leq \beta_0 \leq \hat{\beta}+r_a\widehat{se}_{\beta}$ が信頼度 $1-a$ の信頼区間に他ならない．この信頼区間の振る舞いをモンテカルロ実験を行って理解する．

### データ発生過程（母集団）
$$
y_i = \alpha_0 + \beta_0 x_i + u_i \\
\alpha_0 = 1, ~~ \beta_0 = 0.5, ~~ u_i \overset{i.i.d}{\sim} N(0, 0.02), ~~ x_i \overset{iid}{\sim} U(-0.5,0.5)
$$
この DGP からモンテカルロ標本を $J = 10^5$ 個作成し，それぞれについて推定値 $\hat{\beta}_j, \widehat{se}_{\beta_j}$ を計算し，信頼区間を求める．$J$ この標本から計算した信頼区間のうち何個の標本で真のパラメータ $\beta_0$ を含む区間を推定できたかをカウントし，その割合が信頼度に一致することを確認する．